In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import matplotlib.pyplot as plt
from IPython.display import Image 
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

import team36
from team36.mnist.vgg import VGG
from team36.attacks.fast_gradient_attack_data_set import FastSignGradientAttackDataSet
from team36.training import validate, accuracy, predict

DIR = '.'
DATA_DIR = f'{DIR}/data'

In [20]:
model = VGG()
state_dict = torch.load(f"{DIR}/checkpoints/mnist-vgg.pth", map_location=torch.device('cpu'))
model.load_state_dict(state_dict)

defense_model = VGG()
state_dict = torch.load(f"{DIR}/checkpoints/mnist-vgg-defense.pth", map_location=torch.device('cpu'))
defense_model.load_state_dict(state_dict)

criterion = nn.CrossEntropyLoss()

In [21]:
test_set = torchvision.datasets.MNIST(root=DATA_DIR, train=False, download=True, 
                                      transform=transforms.ToTensor())

test_loader = torch.utils.data.DataLoader(test_set, batch_size=100, shuffle=False, num_workers=2)

test_accuracy, _, test_loss = validate(None, test_loader, model, criterion)
defense_test_accuracy, _, defense_test_loss = validate(None, test_loader, defense_model, criterion)

print(f"Regular Test Accuracy is {test_accuracy}")
print(f"Regular Test Loss is {test_loss}")

print(f"Defense Regular Test Accuracy is {defense_test_accuracy}")
print(f"Defense Regular Test Loss is {defense_test_loss}")

Regular Test Accuracy is 0.9839000105857849
Regular Test Loss is 0.0634659081697464
Defense Regular Test Accuracy is 0.9811000227928162
Defense Regular Test Loss is 0.06829589605331421


In [22]:
attack_test_set = torchvision.datasets.MNIST(root=DATA_DIR, train=False, download=True, 
                                             transform=transforms.ToTensor())
attack_test_set = FastSignGradientAttackDataSet(attack_test_set, model, criterion, 
                                                               epsilon=0.25)

attack_test_loader = torch.utils.data.DataLoader(attack_test_set, batch_size=100, 
                                                 shuffle=False, num_workers=0)

attack_test_accuracy, _, attack_test_loss = validate(None, attack_test_loader, model, criterion)
attack_defense_test_accuracy, _, attack_defense_test_loss = validate(None, attack_test_loader, 
                                                                     defense_model, criterion)


print(f"Attack Test Accuracy is {attack_test_accuracy}")
print(f"Attack Test Loss is {attack_test_loss}")

print(f"Attack Defense Test Accuracy is {attack_defense_test_accuracy}")
print(f"Attack Defense Test Loss is {attack_defense_test_loss}")

Attack Test Accuracy is 0.5267999768257141
Attack Test Loss is 1.452913522720337
Attack Defense Test Accuracy is 0.8931000232696533
Attack Defense Test Loss is 0.34969258308410645


In [23]:
index = 10

input, _ = test_set[index]
display(transforms.functional.to_pil_image(input))
print(predict(model, input))

input, _ = attack_test_set[index]
display(transforms.functional.to_pil_image(input))
print(predict(model, input))

input, _ = attack_test_set[index]
display(transforms.functional.to_pil_image(input))
print(predict(defense_model, input))

0


2


0
